https://uis.fabric-testbed.net/ui/

https://uis.fabric-testbed.net/ui/#/people/people_uuid_get

ssh -i ~/work/fabric_config/pruth_fabric_rsa  -L 2001:localhost:9200  -F ~/work/fabric_config/ssh_config pruth@mfelk-p.fabric-testbed.net

In [ ]:
from fabric_reports_client.reports_api import ReportsApi
from datetime import datetime, timezone
import json
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

api =  ReportsApi(base_url="https://reports.fabric-testbed.net:8443/reports", token_file="/Users/kthare10/work/id_token_prod.json")

query_start=None

query_end = datetime.now(timezone.utc)
query_end = query_end.isoformat(timespec='milliseconds')

## Query Projects

In [ ]:
print("Query All projects")
projects = api.query_projects(fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:
print("Query All projects in a time range")

# NOTE: If no start time is provided, it will default to 30 days before the end time

projects = api.query_projects(end_time=query_end, fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:
print("Query All projects using SmartNIC in a time range")
component_type="SmartNIC"
# NOTE: If no start time is provided, it will default to 30 days before the end time
projects = api.query_projects(end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:

print("Query All projects using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
# NOTE: If no start time is provided, it will default to 30 days before the end time
projects = api.query_projects(end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:
print("Query project using SmartNIC and ConnectX_5 in a time range for a specific project")
# NOTE: If no start time is provided, it will default to 30 days before the end time
projects = api.query_projects(end_time=query_end, fetch_all=False, project_id="990d8a8b-7e50-4d13-a3be-0f133ffa8653")
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

## Query Users

In [ ]:
print("Query All users")
users = api.query_users(fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:
print("Query All users in a time range")
# NOTE: If no start time is provided, it will default to 30 days before the end time
users = api.query_users(end_time=query_end, fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:
print("Query All users using SmartNIC in a time range")
component_type="SmartNIC"
# NOTE: If no start time is provided, it will default to 30 days before the end time
users = api.query_users(end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:

print("Query All users using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
# NOTE: If no start time is provided, it will default to 30 days before the end time
users = api.query_users(end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:
print("Query All users using SmartNIC and ConnectX_5 in a time range for a specific project")
# NOTE: If no start time is provided, it will default to 30 days before the end time
users = api.query_users(end_time=query_end, fetch_all=False, project_id="990d8a8b-7e50-4d13-a3be-0f133ffa8653")
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

## Query Slices

In [ ]:
print("Query All slices")
slices = api.query_slices(fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:
print("Query All slices in a time range")
# NOTE: If no start time is provided, it will default to 30 days before the end time
slices = api.query_slices(end_time=query_end, fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:
print("Query All slices using SmartNIC in a time range")
component_type="SmartNIC"
# NOTE: If no start time is provided, it will default to 30 days before the end time
slices = api.query_slices(end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:

print("Query All slices using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
# NOTE: If no start time is provided, it will default to 30 days before the end time
slices = api.query_slices(end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:
print("Query All slices using SmartNIC and ConnectX_5 in a time range for a specific slice")
# NOTE: If no start time is provided, it will default to 30 days before the end time
slices = api.query_slices(end_time=query_end, fetch_all=False, slice_id="f4bfccc3-1360-4148-90c0-b3f21b9240c2")
print(f"Total Slivers: {slices.get('total')}")
print(json.dumps(slices.get("data"), indent=4))

## Query Slivers

In [ ]:
print("Query All slivers")
slivers = api.query_slivers(fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query VM slivers in a time range")
# NOTE: If no start time is provided, it will default to 30 days before the end time
slivers = api.query_slivers(end_time=query_end, sliver_type=["VM"], fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query VM slivers in a time range excluding CERN and CIEN")
# NOTE: If no start time is provided, it will default to 30 days before the end time
slivers = api.query_slivers(end_time=query_end, sliver_type=["VM"], 
                            exclude_site=["CERN", "CIEN"], fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query All slivers using SmartNIC in a time range")
component_type="SmartNIC"
# NOTE: If no start time is provided, it will default to 30 days before the end time
slivers = api.query_slivers(end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:

print("Query All slivers using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
# NOTE: If no start time is provided, it will default to 30 days before the end time
slivers = api.query_slivers(end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query All slivers using SmartNIC and ConnectX_5 in a time range for a specific slice")
# NOTE: If no start time is provided, it will default to 30 days before the end time
slivers = api.query_slivers(end_time=query_end, fetch_all=False, slice_id="7975644c-516d-4492-9420-a62010efac2d")
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data"), indent=4))

# Plots

## Query all Slices

In [ ]:
# Step 1: Fetch all slices
response = api.query_slices(fetch_all=True, per_page=1000)
slices = response["data"]
print(f"Total slices: {response.get('total')}")
print(json.dumps(slices[0], indent=4))

## Slices per Month

In [ ]:
slices = response["data"]
print(f"Total slices: {len(slices)}")


# Step 3: Create DataFrame and process dates
df = pd.DataFrame(slices) 
print(f"Total slices: {len(df)}")

df['lease_start'] = pd.to_datetime(df['lease_start'], format="mixed", utc=True)
print(f"Total slices: {len(df)}")

print(df['lease_start'].isna().sum())  # How many missing dates


df['month'] = df['lease_start'].dt.to_period('M')  # '2025-04'

print(f"Total slices: {len(df)}")

monthly_counts = df.groupby('month').size().sort_index()
for month, count in monthly_counts.items():
    print(f"{month}: {count}")

# Step 6: Plot
monthly_counts.plot(kind='bar', figsize=(12, 6))
plt.title('Number of Slices Created Per Month')
plt.xlabel('Month')
plt.ylabel('Number of Slices')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()


## Active Slices per Month

In [ ]:
# Create DataFrame
df = pd.DataFrame(slices)

# Robust datetime parsing using 'mixed' format
df['lease_start'] = pd.to_datetime(df['lease_start'], format="mixed", utc=True)
df['lease_end'] = pd.to_datetime(df['lease_end'], format="mixed", utc=True)

# Generate monthly range
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

# Count active slices per month
active_counts = []
for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)
    active = df[(df['lease_start'] <= month_end) & (df['lease_end'] >= month_start)]
    active_counts.append(len(active))

for month, count in zip(month_range, active_counts):
    print(f"{month.strftime('%Y-%m')}: {count}")

# Plot
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), active_counts)
plt.xticks(rotation=45)
plt.title("Active Slices Per Month")
plt.xlabel("Month")
plt.ylabel("Active Slice Count")
plt.tight_layout()
plt.grid(axis='y')
plt.show()


## Active Slices with SmartNICs

In [ ]:
# Step 1: Fetch all slices that used SmartNICs
response = api.query_slices(component_type=["SmartNIC", "GPU"], fetch_all=True, per_page=1000,
                            exclude_project_id=["990d8a8b-7e50-4d13-a3be-0f133ffa8653", "6b76128d-c73f-431f-a245-0397586a7d40"])
print(f"Total slices: {response.get('total')}")
slices = response["data"]
print(f"Total slices: {len(slices)}")


# Step 3: Create DataFrame and process dates
df = pd.DataFrame(slices) 
print(f"Total slices: {len(df)}")

# Step 2: Convert lease_start and lease_end to datetime (with UTC awareness)
# Robust datetime parsing using 'mixed' format
df['lease_start'] = pd.to_datetime(df['lease_start'], utc=True, format='mixed')
df['lease_end'] = pd.to_datetime(df['lease_end'], utc=True, format='mixed')

# Step 3: Define the month range for plotting
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

# Step 4: Count SmartNIC slices active in each month
smartnic_counts = []

for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)

    active_slices = df[
        (df['lease_start'] <= month_end) &
        (df['lease_end'] >= month_start)
    ]
    smartnic_counts.append(len(active_slices))

for month, count in zip(month_range, smartnic_counts):
    print(f"{month.strftime('%Y-%m')}: {count}")

# Step 5: Plot the results
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), smartnic_counts)
plt.xticks(rotation=45)
plt.title("SmartNIC Slice Usage Per Month")
plt.xlabel("Month")
plt.ylabel("Number of Active SmartNIC Slices")
plt.tight_layout()
plt.grid(axis='y')
plt.show()


## Query current Active Slices

In [ ]:
response = api.query_slices(fetch_all=True, per_page=1000, exclude_slice_state=["Closing", "Dead"], exclude_project_id=["990d8a8b-7e50-4d13-a3be-0f133ffa8653", 
                                                                                                                        "6b76128d-c73f-431f-a245-0397586a7d40",
                                                                                                                        "7a5adb91-c4c0-4a1c-8021-7b6c56af196f"])
slices = response["data"]

print(f"Total slices: {len(slices)}")
display(pd.DataFrame(slices))


## Top 10 users for SmartNIC in a time range

In [ ]:
import pandas as pd
from collections import defaultdict
from dateutil import parser

query_end = datetime.now(timezone.utc)
query_end = query_end.isoformat(timespec='milliseconds')

# NOTE: If no start time is provided, it will default to 30 days before the end time

# Load slivers with SmartNICs
slivers = api.query_slivers(end_time=query_end, component_type="GPU", fetch_all=True, per_page=1000)["data"]

In [ ]:
# {user_id: {"usage": float, "user_email": str, "project_id": str, "project_name": str}}
user_usage = {}

for s in slivers:
    user_id = s.get("user_id")
    user_email = s.get("user_email")
    project_id = s.get("project_id")
    project_name = s.get("project_name")
    lease_start = s.get("lease_start")
    lease_end = s.get("lease_end")
    smartnic_count = s.get("components", {}).get("total", 0)

    if not user_id or not lease_start or not lease_end:
        continue

    try:
        start = pd.to_datetime(lease_start, utc=True)
        end = pd.to_datetime(lease_end, utc=True)
        hours = max((end - start).total_seconds() / 3600.0, 0)
    except Exception:
        continue

    usage = smartnic_count * hours

    if user_id not in user_usage:
        user_usage[user_id] = {
            "smartnic_usage_hours": 0,
            "user_email": user_email,
            "project_id": project_id,
            "project_name": project_name
        }

    user_usage[user_id]["smartnic_usage_hours"] += usage

# Convert to DataFrame
usage_df = pd.DataFrame([
    {
        "user_id": uid,
        "user_email": info["user_email"],
        "project_id": info["project_id"],
        "project_name": info["project_name"],
        "smartnic_usage_hours": info["smartnic_usage_hours"]
    }
    for uid, info in user_usage.items()
])

# Sort and display top 10
top_users = usage_df.sort_values("smartnic_usage_hours", ascending=False).head(10)
display(top_users)
